In [464]:
import json
import pandas as pd
import folium

import urllib.request as req

In [950]:
code = pd.read_csv('행정동코드.csv')
data = pd.read_csv('df19.csv')
pop = pd.read_csv('서울시 우리마을가게 상권분석서비스(상권-직장인구).csv',encoding='cp949')
df2= pd.read_csv('상권영역.csv',index_col=False)

code.rename(columns={'H_SDNG_CD(통계청행정동코드)':"통계청행정동코드"},inplace=True)
code.rename(columns={'H_DNG_CD(행자부행정동코드)':"행정동코드"},inplace=True)
code.rename(columns={"DO_NM(시도명)":"시도명"},inplace=True)
code.rename(columns={"CT_NM(시군구명)":"시군구명"},inplace=True)
code.rename(columns={"H_DNG_NM(행정동명)":"행정동명"},inplace=True)


df2 = pd.merge(df2,code, how='left', left_on='ADSTRD_CD', right_on='행정동코드')#code.iloc[:,1:]
df2 = df2.dropna()

pop = pd.merge(pop,df2, how='left', left_on='상권_코드', right_on='TRDAR_CD')
pop = pop.dropna()

pop.columns.unique()

pop['상권_구분_코드_명'].unique()

array(['관광특구', '전통시장', '발달상권', '골목상권'], dtype=object)

In [951]:
pop = pop.query("(상권_구분_코드_명 in ['관광특구','전통시장','발달상권']) & (기준_년월_코드 in [2018,2019,2020])")#"골목상권",

In [952]:
a=pd.DataFrame(pop.groupby(['통계청행정동코드'], as_index=False)['연령대_10_직장_인구_수','연령대_20_직장_인구_수','연령대_30_직장_인구_수'].sum())

a['청년인구수'] =  a['연령대_20_직장_인구_수'].values+a['연령대_30_직장_인구_수'].values # a['연령대_10_직장_인구_수'].values +
a.drop(columns=['연령대_10_직장_인구_수'],axis=1,inplace=True)
a.drop(columns=['연령대_20_직장_인구_수'],axis=1,inplace=True)
a.drop(columns=['연령대_30_직장_인구_수'],axis=1,inplace=True)
# a['청년인구수'] = a1.values+a2.values+a3.values

data = a
data['통계청행정동코드'] = data['통계청행정동코드'].astype(str)

# yong = a[a['CT_NM(시군구명)']=='용산구']['H_SDNG_CD(통계청행정동코드)'].to_list()

# pop = pd.DataFrame(pop.groupby(['통계청행정동코드'], as_index=False)['분기당_매출_금액'].sum())
# pop['통계청행정동코드'] = pop['통계청행정동코드'].astype(int)
# pop['통계청행정동코드'] = pop['통계청행정동코드'].astype(str)

C:\ananconda\envs\multi\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [956]:
top5 = data.sort_values(by=['청년인구수'],ascending=False).iloc[:10]['통계청행정동코드'].to_list()

df2['통계청행정동코드'] = df2['통계청행정동코드'].astype(str)

def top(x):
    if x['통계청행정동코드'] in top5:
        return x['행정동명']

top_5 = pd.DataFrame(df2.apply(top,axis=1))

top_5.dropna(inplace=True)

top_5[0].unique()

array(['종로1.2.3.4가동', '삼성1동', '소공동', '가산동', '대치2동', '서초2동', '역삼1동',
       '삼성2동', '구로3동', '여의동'], dtype=object)

In [957]:
seoul_map = folium.Map(location=[37.5502, 126.982], zoom_start=10.5, tiles='cartodbpositron')
seoul_map.choropleth(geo_data =json_obj,
                    data = data,
                    columns=['통계청행정동코드','청년인구수'],
                    fill_color='PuRd',
                    key_on='properties.code',
                    highlight=True,
                    fill_opacity=0.5, line_opacity=1,
                    legend_name='money'
                    )
folium.LayerControl().add_to(seoul_map)

C:\ananconda\envs\multi\lib\site-packages\folium\folium.py:413: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [955]:
seoul_map